In [55]:
import numpy as np

In [56]:
inputs = [[1, 2, 3, 2.5],
          [2.0,5.0, -1.0, 2.0],
          [-1.5, 2.7, 3.3, -0.8]]
weights1 = [[0.2, 0.8, -0.5, 1.0], 
           [0.5, -0.91, 0.26, -0.5], 
           [-0.26, -0.27, 0.17, 0.87]]
biases1 = [2, 3, 0.5]
weights2 = [[0.1, -0.14, 0.5], 
           [-0.5, 0.12, -0.33], 
           [-0.44, 0.73, -0.13]]
biases2 = [-1, 2, -0.5]

In [57]:
layer1_output = np.dot(inputs, np.array(weights1).T) + biases1
layer2_output = np.dot(layer1_output, np.array(weights2).T) + biases2
print(layer2_output)

[[ 0.5031  -1.04185 -2.03875]
 [ 0.2434  -2.7332  -5.7633 ]
 [-0.99314  1.41254 -0.35655]]


#### ReLU Activation

In [58]:
inputs = [0, 2, -1, 3.3, -2.7, 1,1, 2.2, -100]
output = []

In [59]:
for i in inputs:
    output.append(max(0,i))
print(output)

[0, 2, 0, 3.3, 0, 1, 1, 2.2, 0]


#### Softmax Activation

In [60]:
layer_outputs = [4.8, 1.21, 2.385]

In [61]:
import math
E = math.e

In [62]:
exp_values = []
for output in layer_outputs:
    exp_values.append(E**output)
print(exp_values)

[121.51041751873483, 3.353484652549023, 10.859062664920513]


In [63]:
norm_base = sum(exp_values)
norm_values = []

for value in exp_values:
    norm_values.append(value/ norm_base)
print(norm_values)
print(sum(norm_values))

[0.8952826639572619, 0.024708306782099374, 0.0800090292606387]
0.9999999999999999


In [64]:
layer_outputs = [[4.8, 1.21, 2.385],
                [8.9, -1.81, 0.2],
                [1.41, 1.051, 0.026]]

In [65]:
exp_values = np.exp(layer_outputs)
norm_values = exp_values / np.sum(exp_values, axis=1, keepdims=True)

In [66]:
print(exp_values)
print(norm_values)

[[1.21510418e+02 3.35348465e+00 1.08590627e+01]
 [7.33197354e+03 1.63654137e-01 1.22140276e+00]
 [4.09595540e+00 2.86051020e+00 1.02634095e+00]]
[[8.95282664e-01 2.47083068e-02 8.00090293e-02]
 [9.99811129e-01 2.23163963e-05 1.66554348e-04]
 [5.13097164e-01 3.58333899e-01 1.28568936e-01]]


#### Categorical Cross-Entropy

In [67]:
import math

softmax_output = [0.7, 0.1, 0.2]
target_output = [1, 0, 0]

loss = -(math.log(softmax_output[0])*target_output[0] +
         math.log(softmax_output[1])*target_output[1] +
         math.log(softmax_output[2])*target_output[2])

print(loss)

0.35667494393873245


In [68]:
loss = -math.log(softmax_output[0])
print(loss)

0.35667494393873245


In [69]:
softmax_outputs = np.array([[0.7, 0.1, 0.2],
                            [0.1, 0.5, 0.4],
                            [0.02, 0.9, 0.08]])

class_targets = [0, 1, 1]

print(-np.log(softmax_outputs[[0, 1, 2] , [class_targets]]))

[[0.35667494 0.69314718 0.10536052]]


### Final Structured Approach

In [70]:
from nnfs.datasets import spiral_data
X, y = spiral_data (samples=100, classes=3)

In [71]:
# np.random.seed(0)
# X = [[1, 2, 3, 2.5],
#     [2.0,5.0, -1.0, 2.0],
#     [-1.5, 2.7, 3.3, -0.8]]

In [72]:
class DenseLayer:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1,n_neurons))
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

In [73]:
class ActivationReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

In [74]:
class ActivationSoftMax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

In [75]:
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss

class CategoricalCrossEntropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)

        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods            

In [76]:
dense1 = DenseLayer(2,3)
activation1 = ActivationReLU()
dense2 = DenseLayer(3,3)
activation2 = ActivationSoftMax()

In [77]:
dense1.forward(X)
activation1.forward(dense1.output)

dense2.forward(activation1.output)
activation2.forward(dense2.output)

In [78]:
loss_function = CategoricalCrossEntropy()
loss = loss_function.calculate(activation2.output, y)

In [79]:
print("Loss:", loss)

Loss: 1.098530900186496
